In [ ]:
import numpy as np 
import pandas as pd 
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

In [ ]:
train = pd.read_csv("../input/kk30ml/train.csv")

In [ ]:
train["kfold"] = 0

kflod = KFold(n_splits = 10, shuffle = True, random_state = 0)
for fold, (train_ind, val_ind) in enumerate(kflod.split(X = train)):
     print(fold, train_ind, val_ind)
     train.loc[val_ind, "kfold"] = fold 

In [ ]:

train.to_csv("train_kf.csv", index = False)

In [ ]:
train.kfold.value_counts()

In [ ]:
ttrain = pd.read_csv("./train_kf.csv")
ttest = pd.read_csv("../input/kk30ml/test.csv")
ssub = pd.read_csv("../input/kk30ml/sample_submission.csv")

print(ttrain.shape,ttest.shape)
ttrain.sample()

In [ ]:
useful_feat = [c for c in ttrain.columns if c not in ("id", "target", "kfold")]
catcol = [c for c in useful_feat if 'cat' in c]
numcol = [c for c in useful_feat if 'cont' in c]
ttest = ttest[useful_feat]

In [ ]:
final_pred =[]
for fold in range(10):
    
    X_train = ttrain[ttrain.kfold != fold].reset_index(drop = True)
    X_valid = ttrain[ttrain.kfold == fold].reset_index(drop = True)
    X_test = ttest.copy()
    
    y_train = X_train.target
    y_valid = X_valid.target
    
    X_train = X_train[useful_feat]
    X_valid = X_valid[useful_feat]
    
    ordenco = OrdinalEncoder()
    
    X_train[catcol] = ordenco.fit_transform(X_train[catcol])
    X_valid[catcol] = ordenco.transform(X_valid[catcol])
    X_test[catcol] = ordenco.transform(X_test[catcol])
    
    scaler = StandardScaler()
    X_train[numcol] = scaler.fit_transform(X_train[numcol])
    X_valid[numcol] = scaler.transform(X_valid[numcol])
    X_test[numcol] = scaler.transform(X_test[numcol])
    
    xgb_params = {
        'learning_rate': 0.03628302216953097,
        'subsample': 0.7875490025178,
        'colsample_bytree': 0.11807135201147,
        'max_depth': 3,
        'booster': 'gbtree', 
        'reg_lambda': 0.0008746338866473539,
        'reg_alpha': 23.13181079976304,
        'random_state':40,
        'n_estimators':10000
        
        
    }
    
    model = XGBRegressor(**xgb_params,
                       tree_method='gpu_hist',
                       predictor='gpu_predictor',
                       gpu_id=0)
    model.fit(X_train, y_train, early_stopping_rounds=300,eval_set=[(X_valid,y_valid)],verbose=2000)
    pred = model.predict(X_valid)
    preds = model.predict(X_test)
    final_pred.append(preds)
    print (fold, mean_squared_error(y_valid, pred, squared = False))

In [ ]:
pred_FD = np.mean(np.column_stack(final_pred), axis = 1)

In [ ]:
pred_FD.shape

In [ ]:
ssub.target = pred_FD
ssub

In [ ]:
ssub.to_csv("submissionF.csv", index = False)